In [76]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd


In [132]:
#FSAPLALONS000005003574,6472722,511,27-05-2022
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [129]:
df = pd.read_csv("data.csv")
df.head()

,Loan Account,Principal-M,Rate-of-interest,EMI,Earlier-due-date,New-due-date
0,FSAPLALONS000005000073,212739.43,25.73,9465,7,30
1,FSAPLALONS000005000072,37990.00,26.50,5917,7,30
2,FSAPLALONS000005003574,65652.69,26.73,7396,1,30


In [177]:
part_payment = pd.read_csv("pre-payment.csv")
part_payment['REPAY_EFFECTIVE_DATE'] = pd.to_datetime(part_payment['REPAY_EFFECTIVE_DATE'],format="%d-%m-%Y")
part_payment.head()

,Loan Account,Receipt Number,PART_PREPAY_AMOUNT,REPAY_EFFECTIVE_DATE
0,FSAPLALONS000005000115,6458210,729,2022-04-11
1,FSAPLALONS000005000330,6490591,208,2022-04-30
2,FSAPLALONS000005000407,6574372,866,2022-06-23
3,FSAPLALONS000005000487,6472702,598,2022-04-20
4,FSAPLALONS000005000073,6472722,1000,2022-11-07


In [178]:
part_payment['date'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).day
part_payment['month'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).month
part_payment['year'] = pd.DatetimeIndex(part_payment['REPAY_EFFECTIVE_DATE']).year
part_payment.head(8)

,Loan Account,Receipt Number,PART_PREPAY_AMOUNT,REPAY_EFFECTIVE_DATE,date,month,year
0,FSAPLALONS000005000115,6458210,729,2022-04-11,11,4,2022
1,FSAPLALONS000005000330,6490591,208,2022-04-30,30,4,2022
2,FSAPLALONS000005000407,6574372,866,2022-06-23,23,6,2022
3,FSAPLALONS000005000487,6472702,598,2022-04-20,20,4,2022
4,FSAPLALONS000005000073,6472722,1000,2022-11-07,7,11,2022
5,FSAPLALONS000005000073,6472722,2000,2022-11-08,8,11,2022
6,FSAPLALONS000005003574,6472722,416,2022-09-01,1,9,2022


In [179]:
part_payment_list=part_payment.loc[part_payment['Loan Account']=="FSAPLALONS000005003574",['month','date','PART_PREPAY_AMOUNT']].values.tolist()
print(part_payment_list)

[[9, 1, 416]]


In [180]:
part_payment_dict = {item[0]: [item[1],item[2:]] for item in part_payment_list}
print(part_payment_dict)

{9: [1, [416]]}


In [181]:
part_payment_dict.get(9)

[1, [416]]

In [150]:
sm = 9 
if sm == part_payment_dict.get('9-30'):
    print("summa")

416

In [130]:
principal_list = df['Principal-M'].to_list()
roi_list = df['Rate-of-interest'].to_list()
emi_list = df['EMI'].to_list()
edd_list = df['Earlier-due-date'].to_list()

In [41]:
d = edd_list[0]
sd = pd.to_datetime(f"202204{d}", format='%Y%m%d', errors='ignore')
d = 6
ed = pd.to_datetime(f"202204{d}", format='%Y%m%d', errors='ignore')


In [84]:
principal_list[2]

65653.0

In [85]:
def days360(start_date, end_date, method_eu=False):
    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    start_date.is_leap_year is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360)

In [129]:
end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']

In [89]:
sum(part_payment_dict.get(9))

416

In [217]:
end_of_month_dates = ['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31']
interest_value_list = []

def first_closing_prinicipal(principal, roi, emi):
    first_interest = principal * (roi/1200)
    interest_value_list.append(first_interest)
    print("first interest",round(first_interest, 2))
    principal_apposition = emi - first_interest
    print("first prinicipal appo",round(principal_apposition, 2))
    closing_principal = principal - principal_apposition
    print("first closing prinicipal",round(closing_principal, 2))
    return closing_principal

def next_values(principal, roi, emi, date, sm, em, part_payment_dict):

    if sm <= 6:
        interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}'))
        # print("days360 value",days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')))
        # print("sm, em", sm, em)
        # print("date",date)
        print("interest value", round(interest_value,2))
        interest_value_list.append(interest_value)
        principal_apposition = emi - interest_value
        print("principal apposition", round(principal_apposition,2))
        principal = principal - principal_apposition
        print("closing principal", round(principal,2))
        if sm==6:
            print("closing principal inside june before interest", round(principal,2))
            interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f'2022-6-{date}'), pd.to_datetime(f'2022-6-30'))
            interest_value_list.append(interest_value)
            print("inside june interest", round(interest_value,2))
            principal = principal + interest_value
            print("june closing principal", round(principal,2))
    if sm>6:
        print("closing principal after june before interest", round(principal,2))
        if part_payment_dict.get(sm):
            principal = principal - sum(part_payment_dict.get(sm)[1])
            interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f"{end_of_month_dates[sm-2]}"), pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0]}'))
            print("START DATE",pd.to_datetime(f"{end_of_month_dates[sm-2]}"))
            print("END DATE",pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0]}'))
            print("INTEREST VALUE", round(interest_value),2)
            print("principal after part payment", round(principal,2))

        interest_value = principal * (roi/ 36000) * days360(pd.to_datetime(f"{end_of_month_dates[sm-1]}"), pd.to_datetime(f"{end_of_month_dates[sm]}"))

        # print("days360 value",days360(pd.to_datetime(f'2022-{sm}-{date}'), pd.to_datetime(f'2022-{em}-{date}')))
        # print("sm, em", sm, em)
        # print("date",date)
        print("interest value", round(interest_value,2))
        interest_value_list.append(interest_value)

        principal_apposition = emi - interest_value
        print("principal apposition", round(principal_apposition,2))
        principal = principal - principal_apposition
        print("closing principal", round(principal,2))
        
    return interest_value,principal

In [211]:
end_of_month_dates[7]

'2022-08-31'

In [212]:
sm =9 
pd.to_datetime(f'2022-{sm}-{part_payment_dict.get(sm)[0]}')

Timestamp('2022-09-01 00:00:00')

In [218]:
closing_principal = first_closing_prinicipal(principal_list[2], roi_list[2], emi_list[2])
for i in range(5, 10):
    interest_value, closing_principal_2 = next_values(closing_principal, roi_list[2], emi_list[2], edd_list[2], i, i+1, part_payment_dict)
    closing_principal = closing_principal_2
    # else:
    #     break

first interest 1462.41
first prinicipal appo 5933.59
first closing prinicipal 59719.1
interest value 1330.24
principal apposition 6065.76
closing principal 53653.35
interest value 1195.13
principal apposition 6200.87
closing principal 47452.48
closing principal inside june before interest 47452.48
inside june interest 1021.77
june closing principal 48474.25
closing principal after june before interest 48474.25
interest value 1079.76
principal apposition 6316.24
closing principal 42158.01
closing principal after june before interest 42158.01
interest value 939.07
principal apposition 6456.93
closing principal 35701.08
closing principal after june before interest 35701.08
START DATE 2022-08-31 00:00:00
END DATE 2022-09-01 00:00:00
INTEREST VALUE 26 2
principal after part payment 35285.08
interest value 785.98
principal apposition 6610.02
closing principal 28675.05


In [ ]:
closing_principal = first_closing_prinicipal(principal_list[0], roi_list[0], emi_list[0])
for i in range(5, 10):
    interest_value, closing_principal_2 = next_values(closing_principal, roi_list[0], emi_list[0], edd_list[0], i, i+1)
    closing_principal = closing_principal_2
    # else:
    #     break
print(round(interest_value,2))

In [ ]:
closing_principal = first_closing_prinicipal(principal_list[0], roi_list[0], emi_list[0])
for i in range(5, 10):
    interest_value, closing_principal_2 = next_values(closing_principal, roi_list[0], emi_list[0], edd_list[0], i, i+1, part_payment_dict)
    closing_principal = closing_principal_2
    # else:
    #     break
print(round(interest_value,2))

In [147]:
print(interest_value_list)

[4561.49, 4456.35, 4348.95, 3250.1, 4308.95, 4198.39, 4085.47]


In [152]:
df['interest_april1'] = ""

In [153]:
df['interest_april1'].loc[0] = interest_value_list[0]

/home/laptop-pr-14/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [154]:
df['april_interest'] = ""
df['may_interest'] = ""
df['june_interest'] = ""
df['june_interest'] = ""
df['july_interest'] = ""
df['august_interest'] = ""
df['september_interest'] = ""

df['april_interest1'] = ""
df['may_interest1'] = ""
df['june_interest1'] = ""
df['july_interest1'] = ""
df['august_interest1'] = ""
df['september_interest1'] = ""

,Loan Account,Principal-M,Rate-of-interest,EMI,Earlier-due-date,New-due-date,Date,interest_april,interest_april1
0,FDSG898123,212739.43,25.73,9465,7,30,2022-07-31,4561.49,4561.49
1,FDSG898123,37990.00,26.50,5917,7,30,2022-07-31,4561.49,


In [ ]:

df['interest_april1'].loc[0] = interest_value_list[0]

In [106]:

df['Date'] = pd.to_datetime(f'2022-6-30') + MonthEnd(1)
df['Date'] 

0   2022-07-31
1   2022-07-31
Name: Date, dtype: datetime64[ns]

In [111]:
pd.to_datetime(f'2022-6-23') + MonthEnd(0)

Timestamp('2022-06-30 00:00:00')

In [50]:
first_closing_prinicipal(principal_list[0], roi_list[0], emi_list[0])

4561.49


207835.91794491667

In [51]:
def pre_payment(closing_principal, payment):
    updated_closing_principal =  closing_principal - payment
    return updated_closing_principal


first interest 4561.49
first prinicipal appo 4903.51
first closing prinicipal 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
closing principal inside june before interest 202827.27
inside june interest 3334.2
june closing principal 206161.46
closing principal inside june before interest 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
closing principal inside june before interest 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
closing principal inside june before interest 207835.92
interest value 4456.35
principal apposition 5008.65
closing principal 202827.27
4456.35


In [53]:
principal_list[0]
interest_value = 207835.478725 * (roi_list[0] / 36000) * days360(sd,ed)
print(interest_value)

-148.54463521095138


In [30]:
interest_value = 207835.478725 * (25.73/ 36000) * days360(pd.to_datetime('2022-04-20'), pd.to_datetime('2022-05-20'))
print(interest_value)

4456.339056328541


In [31]:
# decimal - done 
# june  interest value - done
# prepayment logic 
# prepayment interest calculation
# prepayment before june
# multiple prepayment in same month
# sample data verification

In [32]:
days360(sd,ed)

-1

In [33]:
days360(pd.to_datetime('2022-04-20'), pd.to_datetime('2022-05-20'))

30

In [34]:
days360(pd.to_datetime('2012-11-27'), pd.to_datetime('2017-11-27'))

1800